In [1]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException,ElementNotInteractableException
from selenium.webdriver.common.keys import Keys
from urllib.parse import urlencode

In [2]:
base_url="https://www.practo.com/health-checkup-packages/"

In [3]:
chromeOptions = webdriver.ChromeOptions()
prefs={'disk-cache-size': 4096 }
chromeOptions.add_experimental_option('prefs', prefs)
browser = webdriver.Chrome(options=chromeOptions)

In [4]:
def search(location="Delhi",category="diabetes-checkup"):
    print("requesting webpage")
    paramenter={
        "city":location
    }
    encoded_parameter=urlencode(paramenter)
    final_url=f"{base_url}{category}?{encoded_parameter}"
    browser.get(final_url)
    print("webpage opened")

In [38]:
def extract_packages():
    packages=[]
    packages_section=browser.find_elements_by_css_selector('div[id="packages"]>div')
    for package in packages_section:
        package_title = package.find_element_by_css_selector(".c-package__title-box h3").text
        package_desc = package.find_element_by_css_selector(".c-package__title-box p").text
        package_tests = package.find_element_by_css_selector(".c-package__tests-details").text
        package_offer_price = package.find_element_by_css_selector(".c-package__price>span").text
        package_mrp_price = package.find_element_by_css_selector(".c-package__price>s").text
        package_price_discount = package.find_element_by_css_selector(".c-package__price>span:nth-of-type(2)").text
        package_detail={
            "package_title":package_title,
            "package_desc":package_desc,
            "package_tests":package_tests,
            "package_offer_price":package_offer_price,
            "package_mrp_price":package_mrp_price,
            "package_price_discount":package_price_discount,
        }
        packages.append(package_detail)
    return packages

In [44]:
def click_and_collect_data():
    hidden_test_details=[]
    hidden_test_details+=scrape_hidden_test(no_of_test=6)
    i=0
    while True:
        try:
            # clicking next button
            browser.find_element_by_css_selector(".slick-next").click()
            time.sleep(1)
            if i>=6:                
                print("extracting_data")
                hidden_test_details+=scrape_hidden_test()
                i=0            
            i+=1
        except ElementNotInteractableException:
            if i!=0:
                hidden_test_details+=scrape_hidden_test(no_of_test=i)
            break    
    return hidden_test_details
            

In [45]:
def scrape_hidden_test(no_of_test=6):
    details=[]
    hidden_tests=browser.find_elements_by_css_selector("div[class='slick-track'] .slick-active a")
    for test in hidden_tests[6-no_of_test:6]:
        test_data=test.text.split("\n")
        details.append({
            "test_title":test_data[0],

            "test_desc":test_data[1],
            "test_price":test_data[2]
        }) 
    return details

In [46]:
search(location="Delhi",category="diabetes-checkup")
test_result= click_and_collect_data()
package_result = extract_packages()

In [47]:
test_result

[{'test_title': 'HbA1c',
  'test_desc': 'Known as Glycosylated Haemoglobin Blood',
  'test_price': '₹300'},
 {'test_title': 'Insulin Fasting Blood',
  'test_desc': 'Known as Insulin Fasting Blood',
  'test_price': '₹500'},
 {'test_title': 'Fasting Blood Sugar',
  'test_desc': 'Known as Glucose Fasting Blood',
  'test_price': '₹150'},
 {'test_title': 'Lipid Profile',
  'test_desc': 'Known as Lipid Profile Blood',
  'test_price': '₹400'},
 {'test_title': 'Random Blood Sugar',
  'test_desc': 'Known as Glucose Random Blood',
  'test_price': '₹150'},
 {'test_title': 'C Peptide Blood',
  'test_desc': 'Known as C Peptide Random Blood',
  'test_price': '₹500'},
 {'test_title': 'Thyroid Profile (T3, T4, TSH)',
  'test_desc': 'Known as Thyroid Profile Total Blood',
  'test_price': '₹400'},
 {'test_title': 'Post Prandial Blood Sugar',
  'test_desc': 'Known as Glucose Post Prandial Blood',
  'test_price': '₹150'},
 {'test_title': 'Liver Function Test',
  'test_desc': 'Known as Liver Function Tests

In [48]:
package_result

[{'package_title': "Advanced Diabetic Patient's Health Checkup",
  'package_desc': 'Ideal for individuals aged 35 - 85 years',
  'package_tests': '89 tests included',
  'package_offer_price': '₹3599',
  'package_mrp_price': '₹5000',
  'package_price_discount': '28% off'},
 {'package_title': 'Advanced Diabetic Screening Checkup',
  'package_desc': 'Ideal for individuals aged 35 - 85 years',
  'package_tests': '70 tests included',
  'package_offer_price': '₹2199',
  'package_mrp_price': '₹3000',
  'package_price_discount': '27% off'},
 {'package_title': 'Basic Diabetic Screening Checkup',
  'package_desc': 'Ideal for individuals aged 20 - 40 years',
  'package_tests': '66 tests included',
  'package_offer_price': '₹1599',
  'package_mrp_price': '₹2000',
  'package_price_discount': '20% off'},
 {'package_title': "Diabetic Patient's Health Checkup",
  'package_desc': 'Ideal for individuals aged 35 - 85 years',
  'package_tests': '77 tests included',
  'package_offer_price': '₹2799',
  'pac

In [55]:
df=pd.DataFrame(test_result)
df.index=df.index+1
df.to_csv("tests.csv",index=True,index_label='Serial_no')

In [56]:
df

,test_title,test_desc,test_price
1,HbA1c,Known as Glycosylated Haemoglobin Blood,₹300
2,Insulin Fasting Blood,Known as Insulin Fasting Blood,₹500
3,Fasting Blood Sugar,Known as Glucose Fasting Blood,₹150
4,Lipid Profile,Known as Lipid Profile Blood,₹400
5,Random Blood Sugar,Known as Glucose Random Blood,₹150
6,C Peptide Blood,Known as C Peptide Random Blood,₹500
7,"Thyroid Profile (T3, T4, TSH)",Known as Thyroid Profile Total Blood,₹400
8,Post Prandial Blood Sugar,Known as Glucose Post Prandial Blood,₹150
9,Liver Function Test,Known as Liver Function Tests Blood,₹400
10,Complete Blood Picture (CBP),Known as Complete Blood Count Automated Blood,₹300


In [59]:
new=pd.DataFrame(package_result)
new.index=new.index+1
new.to_csv("packages.csv",index=True,index_label='Serial_no')

In [61]:
new

,package_title,package_desc,package_tests,package_offer_price,package_mrp_price,package_price_discount
1,Advanced Diabetic Patient's Health Checkup,Ideal for individuals aged 35 - 85 years,89 tests included,₹3599,₹5000,28% off
2,Advanced Diabetic Screening Checkup,Ideal for individuals aged 35 - 85 years,70 tests included,₹2199,₹3000,27% off
3,Basic Diabetic Screening Checkup,Ideal for individuals aged 20 - 40 years,66 tests included,₹1599,₹2000,20% off
4,Diabetic Patient's Health Checkup,Ideal for individuals aged 35 - 85 years,77 tests included,₹2799,₹4000,30% off


In [62]:
browser.close()